# Process Approved Variants

This notebook:
1. Loads approved variants from the CSV file
2. Copies variants to S3 with new variant IDs (original ID * 100000)
3. Updates the Redshift database
4. Updates the API image list

In [ ]:
# Import the module with all the functions
from process_approved_variants import *

## 1. Load and filter approved images

In [ ]:
# Define the input CSV file
csv_file = "All500Approved.csv"

# Load and filter the approved images
approved_df = load_and_filter_approved_images(csv_file)

# Display the first few rows
approved_df.head()

## 2. Prepare data for Redshift

In [ ]:
# Prepare the data for Redshift
redshift_df = prepare_for_redshift(approved_df)

# Display the first few rows of the prepared data
redshift_df.head()

## 3. Copy variants to S3 with new variant IDs

In [ ]:
# Copy the variants to S3
processed_df = copy_approved_variants_to_s3(redshift_df)

# Display a summary of the processed data
success_count = processed_df['processed_status'].sum()
print(f"Successfully processed {success_count} of {len(processed_df)} variants")

## 4. Upload prepared data to S3 for Redshift

In [ ]:
# Upload the prepared data to S3 for Redshift
s3_url = upload_to_s3_for_redshift(processed_df)

if s3_url:
    print(f"Successfully uploaded data to S3: {s3_url}")
else:
    print("Failed to upload data to S3")

## 5. Copy data from S3 to Redshift

In [ ]:
# Only proceed if we have a valid S3 URL
if s3_url:
    # Copy the data from S3 to Redshift
    success = copy_s3_to_redshift(s3_url)
    
    if success:
        print("Successfully copied data to Redshift")
    else:
        print("Failed to copy data to Redshift")
else:
    print("Skipping Redshift update due to missing S3 URL")

## 6. Update the image list in the API

In [ ]:
# Only proceed if we successfully copied to Redshift
if 'success' in locals() and success:
    # Update the image list in the API
    result = update_image_list()
    
    if result:
        print("Successfully updated the image list in the API")
    else:
        print("Failed to update the image list in the API")
else:
    print("Skipping API update due to failed Redshift update")

## Processing Complete

All approved variant images have been processed, copied to S3, added to Redshift, and the API has been updated.